In [1]:
import pandas as pd
from pathlib import Path

In [2]:
ghclass = Path('../student_data/23sp/portfolio-repo-grades-1674941210.csv')

In [7]:
import re
url_re = re.compile(r'https://students.cs.calvin.edu/.+(\.zip|\.tgz|\.tar.gz)')

In [9]:
moodle_assignments = [x for x in Path('../student_data/23sp/23SP CS-344-A-Homework 1-1575205/').glob('**/onlinetext.html')]
for f in moodle_assignments:
    x = f.read_text()
    if 'http' in x:
        print(x)
    if m := url_re.search(x):
        print(m)

<!DOCTYPE html><html><head><meta charset="UTF-8"></head><body><p dir="ltr" style="text-align: left;">GitHub:&nbsp;https://github.com/Calvin-CS344-23SP/portfolio-repo-ashill1470/blob/main/hw/Homework1/homework1.ipynb</p><p dir="ltr" style="text-align: left;">Dataset:&nbsp;<span style="font-size: 1rem;">https://students.cs.calvin.edu/~ash37/letters.zip</span></p><p dir="ltr" style="text-align: left;"><span style="font-size: 1rem;"><br></span></p><p></p></body></html>
<re.Match object; span=(303, 352), match='https://students.cs.calvin.edu/~ash37/letters.zip>
<!DOCTYPE html><html><head><meta charset="UTF-8"></head><body><p dir="ltr" style="text-align: left;">https://github.com/Calvin-CS344-23SP/portfolio-repo-hlg5/blob/804e6ae8783593e9e90679abfd9e662a8823cea9/homework1.ipynb<br></p></body></html>
<!DOCTYPE html><html><head><meta charset="UTF-8"></head><body><p dir="ltr" style="text-align: left;"><a href="https://github.com/Calvin-CS344-23SP/portfolio-repo-TheBguy87/blob/main/hw/hw1/hw1.ip

Grep those files for students.cs.calvin.edu links.

Download all the links